In [1]:
from longeval.spark import get_spark
from pathlib import Path
from longeval.collection import ParquetCollection

spark = get_spark(cores=8, memory="20g")

root = Path("~/shared/longeval/2025/parquet").expanduser()
train = ParquetCollection(spark, (root / "train").as_posix())
test = ParquetCollection(spark, (root / "test").as_posix())
queries = train.queries.union(test.queries).cache()
queries.printSchema()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/26 05:53:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/26 05:53:25 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in standalone/kubernetes and LOCAL_DIRS in YARN).
25/05/26 05:53:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)
 |-- split: string (nullable = true)
 |-- language: string (nullable = true)
 |-- date: string (nullable = true)



In [4]:
expanded = spark.read.json(
    Path("~/scratch/longeval/query_expansion/expansion").expanduser().as_posix(),
    multiLine=True,
).cache()
expanded.printSchema()

root
 |-- qid: string (nullable = true)
 |-- query: string (nullable = true)



In [5]:
expanded.show()

+-----+--------------------+
|  qid|               query|
+-----+--------------------+
|57776|taxe archeologie ...|
|57777|terre azur, produ...|
|57778|terre de compta, ...|
|57779|terreau semis, co...|
|57780|total energies cl...|
|57781|tram aeroport bor...|
|57782|triploidie, genet...|
|57783|vente bateau semi...|
|57784|verif impots.gouv...|
|57785|vhf bateau, radio...|
|57786|voiture ecologiqu...|
|57787|voiture hybride l...|
|57788|voiture occasion ...|
|57789|voiture occasion ...|
|57790|voiture electriqu...|
|57791|2001 odyssee de l...|
|57794|ajouter qwant au ...|
|57795|aleop loire atlan...|
|57796|aller a minorque ...|
|57797|amicaleloirebreta...|
+-----+--------------------+
only showing top 20 rows


In [6]:
queries.select("qid").distinct().count(), expanded.select("qid").distinct().count()

(67822, 67822)

In [14]:
from pyspark.sql import functions as F

queries.join(
    expanded.selectExpr("qid", "query as expansion"), on="qid", how="inner"
).select(
    "qid",
    F.array_size(F.split("query", " ")),
    F.array_size(F.split("expansion", " ")),
).describe().show()

+-------+------------------+-------------------------------+-----------------------------------+
|summary|               qid|array_size(split(query,  , -1))|array_size(split(expansion,  , -1))|
+-------+------------------+-------------------------------+-----------------------------------+
|  count|            216789|                         216789|                             216789|
|   mean|29130.835494420844|             3.0775454474166124|                  17.89479632269165|
| stddev|21604.085727908263|             2.9017716522035277|                 26.240892518913782|
|    min|                10|                              1|                                  1|
|    max|              9999|                            954|                               2926|
+-------+------------------+-------------------------------+-----------------------------------+

